# core

This is the core and only module of the nbsstorytime library. Outputs the content of all cells (in Jupyter) or messages (in SolveIt) that start with `#| n4l`.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import nbformat
from functools import partial

In [ ]:
def is_n4l_cell(cell):
    """Pure function to check if cell has n4l marker"""
    return (cell.cell_type == 'raw' and 
            cell.source.strip().startswith('#| n4l'))

def extract_n4l_content(cell):
    """Pure function to extract content without marker"""
    lines = cell.source.strip().split('\n')
    return '\n'.join(lines[1:]).strip()

def process_notebook(notebook_path):
    """Main processing function using functional composition"""
    with open(notebook_path, 'r') as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Functional pipeline
    return list(map(extract_n4l_content, 
                   filter(is_n4l_cell, notebook.cells)))


In [ ]:
#| export
import inspect

In [ ]:
#| export
def find_dialog_name():
    for frame_info in inspect.stack():
        if '__dialog_name' in frame_info.frame.f_locals:
            return frame_info.frame.f_locals['__dialog_name']
    return None
    

In [ ]:
from ipykernel import get_connection_info

In [ ]:
# Get full connection details
conn_info = None
try:
    conn_info = get_connection_info()
except RuntimeError as e:
    print("not running in notebook", e)
conn_info

In [ ]:
import json

In [ ]:
#| export
import json
from ipykernel import get_connection_info
from fastcore.basics import Path
from fastcore.basics import patch
import nbformat

In [ ]:
#| export
N4L_SOLVEIT_ROOT = Path('/app/data')

In [ ]:
#| export
def find_current_notebook():
    # Try SolveIt first (via __dialog_name in call stack)
    dname = find_dialog_name()        
    if(dname):
        dname = dname + ".ipynb"
        return N4L_SOLVEIT_ROOT / dname 
    else:    
        # Fall back to Jupyter
        try:
            conn_info = get_connection_info()
            if conn_info:
                conn_data = json.loads(conn_info)
                jupyter_session = conn_data.get('jupyter_session')
                if jupyter_session:
                    return Path(jupyter_session)
        except RuntimeError:
            pass
    
    return None

In [ ]:
nb = None
try:
    nb = find_current_notebook()
except: pass
nb

In [ ]:
#| export
@patch
def export_n4l_content(self:Path):
    """Add n4l export capability to any Path object"""
    # Load the notebook
    notebook = nbformat.read(self.open(), as_version=4)
    
    # Find n4l cells (your existing logic)
    n4l_contents = []
    for cell in notebook.cells:
        if (cell.cell_type == 'raw' and 
            cell.source.strip().startswith('#| n4l')):
            # Remove the marker line
            lines = cell.source.strip().split('\n')
            content = '\n'.join(lines[1:]).strip()
            n4l_contents.append(content)
    
    # Create output file
    output_path = self.with_suffix('.n4l')
    combined_content = '\n\n'.join(n4l_contents)
    output_path.write_text(combined_content)
    
    return output_path

In [ ]:
#| export
def nbsstorytime_export():
    current_path = find_current_notebook()
    if(current_path): 
        current_path.export_n4l_content()
    else:
        print("no current notebook")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

I'd like this code to be usable directly , not only in a Jupyter notebook, but also within SolveIt. As you can see, the ipykernel get_connection_info doesn't return the name of a Jupyter session when running in SolveIt. I suppose that there is an alternative way of finding the name of the current Dialog in SolveIt. You can look at the source code of the module dialoghelper from Answer.AI to find this way. Please suggest how to change the code so that it works in both Jupyter and SolveIt.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Let me look at the dialoghelper source code to find how it gets the current dialog:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01CU3D2uNQ5iFxEmykppjgg3",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://github.com/AnswerDotAI/dialoghelper/blob/main/dialoghelper/core.py"
    }
  },
  "result": "'# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/00_core.ipynb.\\n\\n# %% auto 0\\n__all__ = [\\'md_cls_d\\', \\'dh_settings\\', \\'Placements\\', \\'empty\\', \\'add_styles\\', \\'find_var\\', \\'set_var\\', \\'find_dname\\', \\'find_msg_id\\',\\n           \\'call_endp\\', \\'curr_dialog\\', \\'msg_idx\\', \\'add_scr\\', \\'iife\\', \\'pop_data\\', \\'fire_event\\', \\'event_get\\', \\'find_msgs\\',\\n           \\'add_html\\', \\'read_msg\\', \\'read_msgid\\', \\'add_msg\\', \\'del_msg\\', \\'update_msg\\', \\'run_msg\\', \\'url2note\\', \\'ast_py\\',\\n           \\'ast_grep\\', \\'get_repo\\', \\'msg_insert_line\\', \\'msg_str_replace\\', \\'msg_strs_re<TRUNCATED>"
}
```

</details>

I found it. In `dialoghelper`, there's a `find_dname()` function that looks for `__dialog_name` in the call stack:

```python
def find_dname():
    "Get the message id by searching the call stack for __dialog_id."
    return dh_settings.get('dname', find_var('__dialog_name'))
```

So in SolveIt, the dialog name is available via the `__dialog_name` variable (injected into the execution context).

You can update your `find_current_notebook()` to try both approaches:

```python
def find_current_notebook():
    # Try SolveIt first (via __dialog_name in call stack)
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return Path(dname)
    except: pass
    
    # Fall back to Jupyter
    try:
        conn_info = get_connection_info()
        if conn_info:
            conn_data = json.loads(conn_info)
            jupyter_session = conn_data.get('jupyter_session')
            if jupyter_session:
                return Path(jupyter_session)
    except RuntimeError:
        pass
    
    return None
```

Alternatively, if you don't want to import dialoghelper, you can use the same `find_var` approach directly to look for `__dialog_name` in the call stack.

In [ ]:
#| export
def find_dname():
    "Get the message id by searching the call stack for __dialog_id."
    return dh_settings.get('dname', find_var('__dialog_name'))

because of the way solveit works, the Path returned in find_current_notebook should contain ONLY the filename not the folders before it. Because the library is supposed to be run from Jupyter or SolveIt. Could you fix ?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Just add `.name` to get only the filename:

```python
def find_current_notebook():
    # Try SolveIt first (via __dialog_name in call stack)
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return Path(dname).name
    except: pass
    
    # Fall back to Jupyter
    try:
        conn_info = get_connection_info()
        if conn_info:
            conn_data = json.loads(conn_info)
            jupyter_session = conn_data.get('jupyter_session')
            if jupyter_session:
                return Path(jupyter_session).name
    except RuntimeError:
        pass
    
    return None
```

Note: This returns a string now (from `.name`). If you need it to remain a `Path` object, wrap it: `return Path(Path(dname).name)`.

ok, the drawback is now that the library won't work if not run from the same directory as the dialog/notebook... Could there be a cleaner way to handle this Path problem ? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good point. A cleaner approach would be to keep the full path but resolve it relative to the current working directory when needed:

```python
def find_current_notebook():
    # Try SolveIt first
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return Path(dname).resolve()
    except: pass
    
    # Fall back to Jupyter
    try:
        conn_info = get_connection_info()
        if conn_info:
            conn_data = json.loads(conn_info)
            jupyter_session = conn_data.get('jupyter_session')
            if jupyter_session:
                return Path(jupyter_session).resolve()
    except RuntimeError:
        pass
    
    return None
```

Using `.resolve()` gives you the absolute path, so it works from any directory. Then your `export_n4l_content` function will work correctly regardless of where the script is run from.

Does SolveIt's `find_dname()` return an absolute path or a relative one?

As you can see it returns a relative path

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Got it. So `.resolve()` should work then — it will convert `'nbsstorytime/nbs/00_core'` to an absolute path based on the current working directory.

The only case where this fails is if someone changes directories after starting the notebook/dialog. Is that a use case you need to support?

If so, one option is to capture the absolute path once at import time:

```python
_INITIAL_CWD = Path.cwd()

def find_current_notebook():
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return (_INITIAL_CWD / dname).resolve()
    except: pass
    # ... rest of function
```

No I will not cover this edge case

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Makes sense. Then `.resolve()` is the clean solution — it handles both relative (SolveIt) and potentially absolute (Jupyter) paths correctly.

It is not working : see that after using resolve my path becomes too long and is wrong. Please fix your solution.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

I see the issue — the CWD already includes part of the path, so `.resolve()` doubles it.

Simple fix: don't use `.resolve()` for SolveIt, just return the relative path directly:

```python
def find_current_notebook():
    # Try SolveIt first
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return Path(dname)
    except: pass
    
    # Fall back to Jupyter
    try:
        conn_info = get_connection_info()
        if conn_info:
            conn_data = json.loads(conn_info)
            jupyter_session = conn_data.get('jupyter_session')
            if jupyter_session:
                return Path(jupyter_session).resolve()
    except RuntimeError:
        pass
    
    return None
```

SolveIt's relative path should work as-is if you're running from the expected directory.

There is still a bug. when calling the code from "nbsstorytime/nbs/index.ipynb" within SolveIt it doesn't find the relative path nbsstorytime/nbs/index because we are already running in the folder nbsstorytime/nbs/ . So in the case of SolveIt, maybe we should be working with absolute paths ?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Ah, I see — `find_dname()` returns a path relative to SolveIt's data root (`/app/data/`), not the current working directory.

Try using the absolute SolveIt root:

```python
def find_current_notebook():
    # Try SolveIt first
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return Path('/app/data') / dname
    except (ImportError, ValueError): pass
    
    # Fall back to Jupyter
    try:
        conn_info = get_connection_info()
        if conn_info:
            conn_data = json.loads(conn_info)
            jupyter_session = conn_data.get('jupyter_session')
            if jupyter_session:
                return Path(jupyter_session).resolve()
    except RuntimeError:
        pass
    
    return None
```

Hardcoding `/app/data` is a bit fragile though. Is there a dialoghelper function or environment variable that provides the SolveIt data root?

Please look at the dialoghelper source code and let me know if you find it.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>

```json
{
  "id": "toolu_01UQ34uVFiNPP4wzPTfH7sYk",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://raw.githubusercontent.com/AnswerDotAI/dialoghelper/main/dialoghelper/core.py"
    }
  },
  "result": "'# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/00_core.ipynb.\\n\\n# %% auto 0\\n__all__ = [\\'md_cls_d\\', \\'dh_settings\\', \\'Placements\\', \\'empty\\', \\'add_styles\\', \\'find_var\\', \\'set_var\\', \\'find_dname\\', \\'find_msg_id\\',\\n           \\'call_endp\\', \\'curr_dialog\\', \\'msg_idx\\', \\'add_scr\\', \\'iife\\', \\'pop_data\\', \\'fire_event\\', \\'event_get\\', \\'find_msgs\\',\\n           \\'add_html\\', \\'read_msg\\', \\'read_msgid\\', \\'add_msg\\', \\'del_msg\\', \\'update_msg\\', \\'run_msg\\', \\'url2note\\', \\'ast_py\\',\\n           \\'ast_grep\\', \\'get_repo\\', \\'msg_insert_line\\', \\'msg_str_replace\\', \\'msg_strs_re<TRUNCATED>"
}
```

</details>

I don't see a constant for the SolveIt data root in the dialoghelper source. However, I notice that SolveIt always uses `/app/data/` as the base path.

A clean solution would be to check if we're in SolveIt and prepend the known root:

```python
SOLVEIT_ROOT = Path('/app/data')

def find_current_notebook():
    # Try SolveIt first
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return SOLVEIT_ROOT / dname
    except: pass
    
    # Fall back to Jupyter
    try:
        conn_info = get_connection_info()
        if conn_info:
            conn_data = json.loads(conn_info)
            jupyter_session = conn_data.get('jupyter_session')
            if jupyter_session:
                return Path(jupyter_session).resolve()
    except RuntimeError:
        pass
    
    return None
```

The `/app/data` path appears to be a SolveIt convention. You could also make it configurable via an environment variable if you want more flexibility.

Couldn't you return a Path object instead of a raw "return SOLVEIT_ROOT / dname" ? It would cleaner

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

`SOLVEIT_ROOT / dname` already returns a `Path` object, since `SOLVEIT_ROOT` is defined as `Path('/app/data')`. The `/` operator on a `Path` returns a new `Path`.

So you're already getting a `Path` back — no change needed!

when running nbdev_prepare on this library, it fails with the following error:
While Executing Cell #12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/nbdev/test.py", line 52, in test_nb
    k.run_all(nb, exc_stop=True, preproc=_no_eval)
  File "/usr/local/lib/python3.12/site-packages/execnb/shell.py", line 254, in run_all
    if self.exc and exc_stop: raise self.exc from None
                              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3701, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-e85db143ac30>", line 2, in <module>
    dname = find_dname()
            ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/dialoghelper/core.py", line 73, in find_dname
    return dh_settings.get('dname', find_var('__dialog_name'))
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/dialoghelper/core.py", line 61, in find_var
    return _find_frame_dict(var)[var]
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/dialoghelper/core.py", line 57, in _find_frame_dict
    raise ValueError(f"Could not find {var} in any scope")
ValueError: Could not find __dialog_name in any scope

It is as if we cannot use find_dname outside of SolveIt. How could we fix this ?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The issue is that `find_dname()` raises a `ValueError` when `__dialog_name` isn't found. Your code needs to catch that:

```python
def find_current_notebook():
    # Try SolveIt first
    try:
        from dialoghelper import find_dname
        dname = find_dname()
        if dname: return SOLVEIT_ROOT / dname
    except (ImportError, ValueError): pass
    
    # Fall back to Jupyter...
```

Adding `ValueError` to the exception tuple should fix it.

I don't want to import dialoghelper, could you show how to use the find_var approach directly to look for __dialog_name in the call stack ? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

Here's the approach using `inspect` to search the call stack:

```python
import inspect

def find_dialog_name():
    for frame_info in inspect.stack():
        if '__dialog_name' in frame_info.frame.f_locals:
            return frame_info.frame.f_locals['__dialog_name']
    return None
```

Then use it in `find_current_notebook()`:

```python
def find_current_notebook():
    # Try SolveIt first
    dname = find_dialog_name()
    if dname: return SOLVEIT_ROOT / dname
    
    # Fall back to Jupyter...
```